In [ ]:
!pip install transformers[torch]
!pip install adapters
!pip install datasets

### 🗃️ Dataset Preparation
**The Choice Of Plausible Alternatives** (COPA) is a dataset for commonsense causal reasoning. It consists of questions where each question has a premise and two alternatives and the task is to select the correct alternative given the premise.

In [1]:
from datasets import load_dataset
# Loading the COPA dataset
dataset_en = load_dataset("super_glue", "copa")
dataset_en.num_rows

{'train': 400, 'validation': 100, 'test': 500}

In [2]:
dataset_en['train'][42]

{'premise': 'The executive decided not to hire the applicant.',
 'choice1': 'The applicant failed a background check.',
 'choice2': 'The applicant had experience for the job.',
 'question': 'cause',
 'idx': 42,
 'label': 0}

In [3]:
class DatasetPreprocessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    """Encodes a batch of input data."""
    def encode_batch(self, examples):
      all_encoded = {"input_ids": [], "attention_mask": []}
      for premise, question, choice1, choice2 in zip(examples["premise"], examples["question"], examples["choice1"], examples["choice2"]):
        sentences_a = [premise + " " + question for _ in range(2)]
        # Both answer choices are passed in an array according to the format needed for the multiple-choice prediction head
        sentences_b = [choice1, choice2]
        encoded = self.tokenizer(
            sentences_a,
            sentences_b,
            max_length=60,
            truncation=True,
            padding="max_length",
        )
        all_encoded["input_ids"].append(encoded["input_ids"])
        all_encoded["attention_mask"].append(encoded["attention_mask"])
      return all_encoded
        
    """Encodes input data"""
    def preprocess_dataset(self, dataset):
      dataset = dataset.map(self.encode_batch, batched=True)
      # The transformers model expects the target class column to be named "labels".
      dataset = dataset.rename_column("label", "labels")
      # Transform to pytorch tensors and only output the required columns.
      dataset.set_format(columns=["input_ids", "attention_mask", "labels"])
      return dataset

# Defining the tokenizer and pre-processing the data
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
dp = DatasetPreprocessor(tokenizer)
dataset_en = dp.preprocess_dataset(dataset_en)

### ⁉️ Task 1
1. **Load** the `xlm-roberta-base` model.
2. **Find** language adapters for English and Swahili on the AdapterHub: [https://adapterhub.ml/explore/text_lang/](https://adapterhub.ml/explore/text_lang/).

   Don't forget to make sure when selecting the adapter that it is compatible with the base model. Then load the selected adapters into the base model.
4. **Add** the task adapter and the multiple choice head. You can refer to the following documentation for this: [https://docs.adapterhub.ml/](https://docs.adapterhub.ml/).
5. **Set** the task adapter in the training mode and stack the language adapters (first language, then task adapter).

In [4]:
from adapters.composition import Stack
from transformers import AutoConfig
from adapters import AutoAdapterModel, AdapterConfig

# Solution for Task 1

config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
)
model = AutoAdapterModel.from_pretrained(
    "xlm-roberta-base",
    config=config,
)

# Loading the language adapters
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
model.load_adapter("sw/wiki@ukp", config=lang_adapter_config)

# Adding the task adapter
model.add_adapter("copa")

# Adding a classification head for the target task
model.add_multiple_choice_head("copa", num_choices=2)

# Setting up the adapters
model.train_adapter(["copa"])
model.active_adapters = Stack("en", "copa")

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 🚀 Training

In [5]:
from transformers import TrainingArguments
from adapters import AdapterTrainer
from datasets import concatenate_datasets

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

train_dataset = concatenate_datasets([dataset_en["train"], dataset_en["validation"]])

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


TrainOutput(global_step=160, training_loss=0.6954256534576416, metrics={'train_runtime': 35.5208, 'train_samples_per_second': 140.762, 'train_steps_per_second': 4.504, 'total_flos': 369900586800000.0, 'train_loss': 0.6954256534576416, 'epoch': 10.0})

### ✅ Evaluation
**XCOPA** is a multilingual dataset for causal commonsense reasoning (similarly to COPA) in 11 languages. It includes some resource-poor languages like Eastern Apurímac Quechua and Haitian Creole. In this example we will work with Swahili.

In [6]:
# Loading the XCOPA dataset for Swahili
dataset_sw = load_dataset("xcopa", "sw", verification_mode=None)
print(dataset_sw["test"][5])
dataset_sw = dp.preprocess_dataset(dataset_sw)

{'premise': 'Msichana alipungua nguvu.', 'choice1': 'Alicheza drafti.', 'choice2': 'Aliruka kamba.', 'question': 'cause', 'label': 1, 'idx': 5, 'changed': False}


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### ⁉️ Task 2
1. **Swap** the language adapter for English with the adapter for Sawhili. Evaluate the model using the stacked adapters (execute the cell below). 
2. **Remove** the language adapter and evauate the results using only the task adapter.
3. (Optional) Experiment with a **different language** (e.g., XCOPA supports Turkish, Tamil, Mandarin Chinese etc., you can find pre-trained adapters on [https://adapterhub.ml/explore](https://adapterhub.ml/explore))
4. (Optional) Experiment with a **different model** (e.g., replace `xlm-roberta-base` with `bert-base-multilingual-cased`) and play around with different hyperparameters (learning rate, number of epochs etc.)

In [7]:
# Solution for Task 2
# (1) Stack Swahili and Copa adapters
model.active_adapters = Stack("sw", "copa")

In [9]:
# (2) Remove the language adapter
model.active_adapters = ["copa"]

In [8]:
import numpy as np
from transformers import EvalPrediction

print("Active adapters:", model.active_adapters)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=dataset_sw["test"],
    compute_metrics=compute_accuracy,
)
eval_trainer.evaluate()

Active adapters: Stack[sw, copa]


{'eval_loss': 0.6927564740180969,
 'eval_acc': 0.518,
 'eval_runtime': 2.4518,
 'eval_samples_per_second': 203.933,
 'eval_steps_per_second': 25.695}